In [ ]:
"""Data Loaded Donot execute again"""

import pandas as pd
from sqlalchemy import create_engine, text
import time

db_user = "deepakachyutha"
db_password = "" 
db_host = "localhost"
db_port = "5432"
db_name = "uscrashdata"
csv_file = "US_Accidents_March23.csv" 

print(f"Connecting to {db_name}...")
connection_str = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_str)

with engine.connect() as conn:
        print("Connection successful.")
        
        print("Creating table schema (forcing Zipcode to Text).")

        df_head = pd.read_csv(csv_file, nrows=100, dtype={'Zipcode': str})

        df_head.columns = [c.replace(' ', '_').replace('.', '').replace('(', '').replace(')', '') 
                           for c in df_head.columns]

        df_head.to_sql('raw_accidents', engine, if_exists='replace', index=False)
        print("Table 'raw_accidents' created with correct text columns.")

        print("Importing 7.7 Million rows.")
        start_time = time.time()
        
        raw_conn = conn.connection
        cur = raw_conn.cursor()
        
        with open(csv_file, 'r') as f:
            next(f)
            cur.copy_expert("COPY raw_accidents FROM STDIN WITH CSV", f)
            
        raw_conn.commit()
        print(f"Completed! Loaded in {time.time() - start_time:.1f} seconds.")



Connecting to uscrashdata...
Connection successful.
Creating table schema (forcing Zipcode to Text).
Table 'raw_accidents' created with correct text columns.
Importing 7.7 Million rows.

 ERROR:
COPY from stdin failed: error in .read() call
CONTEXT:  COPY raw_accidents, line 306778

